In [2]:
cd ..

/home/pierreo/semisupervised


In [3]:
import numpy as np
from typing import Callable, Optional, Tuple
from module.quadrature import Quadrature, QuadratureExploration, QuadratureExplorationBis
from botorch import fit_gpytorch_mll
from botorch.models import SingleTaskGP
import torch
from gpytorch.mlls import ExactMarginalLogLikelihood
from torch.distributions.multivariate_normal import MultivariateNormal
import matplotlib.pyplot as plt
import scipy.stats as stats
from gpytorch.kernels import RBFKernel
from gpytorch.kernels.scale_kernel import ScaleKernel
from gpytorch.priors.torch_priors import GammaPrior
import geoopt


/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:

quad_distrib = MultivariateNormal(torch.tensor([1., 0.5]), torch.diag(torch.tensor([0.2, 0.2])))
train_X = torch.linspace(-3,3, 10).reshape(5,2)
train_Y = (-(train_X)**2 + 1.).sum(dim=1, keepdim=True)
covar_module = ScaleKernel(
                RBFKernel(
                    ard_num_dims=train_X.shape[-1],
                    batch_shape=None,
                    lengthscale_prior=GammaPrior(3.0, 6.0),
                ),
                batch_shape=None,
                outputscale_prior=GammaPrior(2.0, 0.15),
            )

model = SingleTaskGP(train_X, train_Y, covar_module=covar_module)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)

quad = Quadrature(model=model,
            distribution=quad_distrib)

/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/botorch/models/gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


# Compare gradient analytic vs. autodiff vs. geoopt autodiff euclidean vs. geoopt autodiff manifold

In [11]:
## autodiff
mean_autodiff, covariance_autodiff = torch.tensor([1., 0.5], requires_grad=True), torch.tensor([[0.2, 0.], [0., 0.2]], requires_grad=True)
m, v = quad._quadrature(mean_autodiff, covariance_autodiff)
m.backward()


## Analytic
quad.gradient_direction()

### Manifold spd gradients
euclidean = geoopt.manifolds.Euclidean()
spd = geoopt.manifolds.SymmetricPositiveDefinite()
manifold = geoopt.manifolds.ProductManifold((euclidean, 2), (spd, (2,2)))

t = torch.zeros(6)
t[0], t[1], t[2], t[5] = 1., 0.5, 0.2, 0.2
t.requires_grad = True
mani_tensor = geoopt.ManifoldTensor(t, manifold=manifold, requires_grad=True)
m, v = quad._quadrature(manifold.take_submanifold_value(mani_tensor, 0), manifold.take_submanifold_value(mani_tensor, 1))
m.backward()

### Manifold euclidean gradients
euclidean = geoopt.manifolds.Euclidean()
manifold = geoopt.manifolds.ProductManifold((euclidean, 2), (euclidean, (2,2)))

t = torch.zeros(6)
t[0], t[1], t[2], t[5] = 1., 0.5, 0.2, 0.2
t.requires_grad = True
mani_tensor_euclidean = geoopt.ManifoldTensor(t, manifold=manifold, requires_grad=True)
m, v = quad._quadrature(manifold.take_submanifold_value(mani_tensor_euclidean, 0), manifold.take_submanifold_value(mani_tensor_euclidean, 1))
m.backward()


print("Gradient autodiff pytorch:")
print("mu:", mean_autodiff.grad)
print("epsilon:", covariance_autodiff.grad)

print("Gradient analytic:")
print("mu:", quad.d_mu)
print("epsilon:", quad.d_epsilon)

print("Gradient autodiff manifold spd geoopt:")
print("mu:", manifold.take_submanifold_value(mani_tensor.grad, 0))
print("epsilon:", manifold.take_submanifold_value(mani_tensor.grad, 1))

print("Gradient autodiff pytorch:")
print("mu:", manifold.take_submanifold_value(mani_tensor_euclidean.grad, 0))
print("epsilon:", manifold.take_submanifold_value(mani_tensor_euclidean.grad, 1))

Gradient autodiff pytorch:
mu: tensor([-0.3725,  0.2864])
epsilon: tensor([[0.5159, 0.0997],
        [0.0997, 0.3543]])
Gradient analytic:
mu: tensor([-0.3725,  0.2864])
epsilon: tensor([[0.5159, 0.0997],
        [0.0997, 0.3543]])
Gradient autodiff manifold spd geoopt:
mu: tensor([-0.3725,  0.2864])
epsilon: tensor([[0.5159, 0.0997],
        [0.0997, 0.3543]])
Gradient autodiff pytorch:
mu: tensor([-0.3725,  0.2864])
epsilon: tensor([[0.5159, 0.0997],
        [0.0997, 0.3543]])


In [17]:
## autodiff
mean_autodiff, covariance_autodiff = torch.tensor([1., 0.5], requires_grad=True), torch.tensor([[0.2, 0.], [0., 0.2]], requires_grad=True)
m, v = quad._quadrature(mean_autodiff, covariance_autodiff)
m.backward()


In [18]:
covariance_autodiff.grad

tensor([[0.5159, 0.0997],
        [0.0997, 0.3543]])

## Test gradient quadrature with vs. without manifold structure input space

In [78]:
### Manifold gradients
euclidean = geoopt.manifolds.Euclidean()
spd = geoopt.manifolds.SymmetricPositiveDefinite()
manifold = geoopt.manifolds.ProductManifold((euclidean, 2), (spd, (2,2)))

t = torch.zeros(6)
t[2], t[5] = .1, .1
t.requires_grad = True

mani_tensor = geoopt.ManifoldTensor(t, manifold=manifold)
param = geoopt.ManifoldParameter(mani_tensor)

In [79]:
m, v = quad._quadrature(manifold.take_submanifold_value(param, 0), manifold.take_submanifold_value(param, 1))

In [80]:
m.backward()

In [81]:
param.grad

tensor([-2.1763,  2.1762, -1.5402, -1.7121, -1.7121, -1.5402])

In [82]:
### Euclidean vanilla gradient
t = torch.zeros(6)
t[2], t[5] = .1, .1
t.requires_grad = True

mani_tensor = geoopt.ManifoldTensor(t)
param = geoopt.ManifoldParameter(mani_tensor)

m, v = quad._quadrature(manifold.take_submanifold_value(param, 0), manifold.take_submanifold_value(param, 1))
m.backward()
param.grad

param.grad.zero_()
m, v = quad._quadrature(manifold.take_submanifold_value(param, 0), manifold.take_submanifold_value(param, 1))
m.backward()
param.grad

tensor([-2.1763,  2.1762, -1.5402, -1.7121, -1.7121, -1.5402])

In [67]:
#### Comparison with implementation


quad.quadrature()
quad.gradient_direction()


In [68]:
quad.d_epsilon

tensor([[-0.1402,  0.0328],
        [ 0.0328, -0.1402]])

In [71]:
param.grad.zero_()

tensor([0., 0., 0., 0., 0., 0.])

In [72]:
param.grad

tensor([0., 0., 0., 0., 0., 0.])

## 1d

In [8]:

quad_distrib = MultivariateNormal(torch.tensor([1.]), torch.diag(torch.tensor([0.2])))
train_X = torch.linspace(-3,3, 10).reshape(-1,1)
train_Y = (-(train_X)**2 + 1.).sum(dim=1, keepdim=True)
covar_module = ScaleKernel(
                RBFKernel(
                    ard_num_dims=train_X.shape[-1],
                    batch_shape=None,
                    lengthscale_prior=GammaPrior(3.0, 6.0),
                ),
                batch_shape=None,
                outputscale_prior=GammaPrior(2.0, 0.15),
            )

model = SingleTaskGP(train_X, train_Y, covar_module=covar_module)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)

quad = Quadrature(model=model,
            distribution=quad_distrib)

/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/botorch/models/gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


In [9]:
## autodiff
mean_autodiff, covariance_autodiff = torch.tensor([1.], requires_grad=True), torch.tensor([[0.2]], requires_grad=True)
m, v = quad._quadrature(mean_autodiff, covariance_autodiff)
m.backward()


## Analytic
quad.gradient_direction()

### Manifold spd gradients
euclidean = geoopt.manifolds.Euclidean()
spd = geoopt.manifolds.SymmetricPositiveDefinite()
manifold = geoopt.manifolds.ProductManifold((euclidean, 1), (spd, (1,1)))

t = torch.zeros(2)
t[0], t[1]= 1., 0.2
t.requires_grad = True
mani_tensor = geoopt.ManifoldTensor(t, manifold=manifold, requires_grad=True)
m, v = quad._quadrature(manifold.take_submanifold_value(mani_tensor, 0), manifold.take_submanifold_value(mani_tensor, 1))
m.backward()

### Manifold euclidean gradients
euclidean = geoopt.manifolds.Euclidean()
manifold = geoopt.manifolds.ProductManifold((euclidean, 1), (euclidean, (1,1)))

t = torch.zeros(2)
t[0], t[1]= 1., 0.2
t.requires_grad = True
mani_tensor_euclidean = geoopt.ManifoldTensor(t, manifold=manifold, requires_grad=True)
m, v = quad._quadrature(manifold.take_submanifold_value(mani_tensor_euclidean, 0), manifold.take_submanifold_value(mani_tensor_euclidean, 1))
m.backward()


print("Gradient autodiff pytorch:")
print("mu:", mean_autodiff.grad)
print("epsilon:", covariance_autodiff.grad)

print("Gradient analytic:")
print("mu:", quad.d_mu)
print("epsilon:", quad.d_epsilon)

print("Gradient autodiff manifold spd geoopt:")
print("mu:", manifold.take_submanifold_value(mani_tensor.grad, 0))
print("epsilon:", manifold.take_submanifold_value(mani_tensor.grad, 1))

print("Gradient autodiff pytorch:")
print("mu:", manifold.take_submanifold_value(mani_tensor_euclidean.grad, 0))
print("epsilon:", manifold.take_submanifold_value(mani_tensor_euclidean.grad, 1))

Gradient autodiff pytorch:
mu: tensor([-1.9981])
epsilon: tensor([[-0.9913]])
Gradient analytic:
mu: tensor([-1.9981])
epsilon: tensor([[-0.9913]])
Gradient autodiff manifold spd geoopt:
mu: tensor([-1.9981])
epsilon: tensor([[-0.9913]])
Gradient autodiff pytorch:
mu: tensor([-1.9981])
epsilon: tensor([[-0.9913]])


In [84]:

quad_distrib = MultivariateNormal(torch.tensor([0.]), torch.diag(torch.tensor([1.])))
train_X = torch.linspace(-3,3, 10).reshape(-1,1)
train_Y = (-(train_X)**2 + 1.).sum(dim=1, keepdim=True)
covar_module = ScaleKernel(
                RBFKernel(
                    ard_num_dims=train_X.shape[-1],
                    batch_shape=None,
                    lengthscale_prior=GammaPrior(3.0, 6.0),
                ),
                batch_shape=None,
                outputscale_prior=GammaPrior(2.0, 0.15),
            )

model = SingleTaskGP(train_X, train_Y, covar_module=covar_module)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)

quad = Quadrature(model=model,
            distribution=quad_distrib)

/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/botorch/models/gpytorch.py:129: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(_get_single_precision_warning(X.dtype), UserWarning)
/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/home/pierreo/miniconda3/envs/semisupervised/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


In [86]:
## Test gradient quadrature with vs. without manifold structure input space
### Manifold gradients
euclidean = geoopt.manifolds.Euclidean()
spd = geoopt.manifolds.SymmetricPositiveDefinite()
manifold = geoopt.manifolds.ProductManifold((euclidean, 1), (spd, (1,1)))

t = torch.zeros(2)
t[1] = .1
t.requires_grad = True

mani_tensor = geoopt.ManifoldTensor(t, manifold=manifold)
param = geoopt.ManifoldParameter(mani_tensor)
m, v = quad._quadrature(manifold.take_submanifold_value(param, 0), manifold.take_submanifold_value(param, 1))
m.backward()
param.grad



tensor([ 6.1035e-05, -1.0131e+00])

In [87]:

quad.quadrature()
quad.gradient_direction()
quad.d_mu, quad.d_epsilon

(tensor([0.]), tensor([[-0.9972]]))

In [ ]:
### Euclidean vanilla gradient
t = torch.zeros(6)
t[2], t[5] = .1, .1
t.requires_grad = True

mani_tensor = geoopt.ManifoldTensor(t)
param = geoopt.ManifoldParameter(mani_tensor)

m, v = quad._quadrature(manifold.take_submanifold_value(param, 0), manifold.take_submanifold_value(param, 1))
m.backward()
param.grad

param.grad.zero_()
m, v = quad._quadrature(manifold.take_submanifold_value(param, 0), manifold.take_submanifold_value(param, 1))
m.backward()
param.grad